<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/MLflow-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

# Exemplo 1

In [7]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/parkinsons.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, stratify=y) # 80% treino e 20% teste

In [ ]:
from sklearn.metrics import confusion_matrix
import mlflow
import mlflow.sklearn

exp = mlflow.set_experiment(experiment_name="experimento_1")

with mlflow.start_run(experiment_id=exp.experiment_id):
    model = tree.DecisionTreeClassifier(criterion="entropy",max_depth=4)
    model = model.fit(X_train, y_train)

    result = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(result, y_test).ravel()

    acc = metrics.accuracy_score(result, y_test)
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    specificity = tn / (tn+fp)
    f1_score = (2 * (precision * recall)) / (precision + recall)

    print("Acurácia: %s" % acc)

    mlflow.log_param("Criterion", "entropy")
    mlflow.log_param("MAX_DEPTH", 4)

    mlflow.log_metric("Acurácia", acc)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("Specificity", specificity)
    mlflow.log_metric("F1-Score", f1_score)

    mlflow.sklearn.log_model(model, "tree")

    get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

# Uso do NGROK

O Ngrok é uma ferramenta que permite criar um túnel seguro para conectar um servidor local a um servidor remoto, tornando possível expor uma aplicação local para a Internet. Essa ferramenta é muito útil para desenvolvedores que precisam testar suas aplicações localmente antes de colocá-las em produção.

In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2fTc6xBFGHTFjXcla4jSNCgSzt4_78Y6vhwwzuAdc7WFxhTfu"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Endereço do MLflow:", ngrok_tunnel.public_url)